In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# Import Feature Engineered Sales Transaction file
df = pd.read_csv('olist_processado.csv')

In [2]:
df.head(2)

,customer_id,customer_city,customer_state,order_id,order_purchase_timestamp,order_item_id,product_id,price,product_category_name,customer_lat,customer_long
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1,a9516a079e37a9c9c36b9b78b10169e8,124.99,moveis_escritorio,-20.500253,-47.400367
1,8912fc0c3bbf1e2fbf35819e21706718,santarem,PA,c1d2b34febe9cd269e378117d6681172,2017-11-09 00:50:13,1,a9516a079e37a9c9c36b9b78b10169e8,112.99,moveis_escritorio,-2.441535,-54.720801


## Top Categorias Vendidas

In [3]:
# Encontrando o número de unidades vendidas de cada categoria de produto
top_itens_df = df.groupby(['product_category_name', 'customer_city', 'customer_state']).agg({'price':'sum'})

In [4]:
top_itens_df.reset_index(inplace=True)

In [5]:
# Rank das categorias mais vendidas
top_itens_df['rank_top_vendidos'] = top_itens_df['price'].rank(method='min',ascending=False).astype(int)

In [6]:
# Lista os top 20 
top_itens_df.sort_values('price',ascending=False).head(20)

,product_category_name,customer_city,customer_state,price,rank_top_vendidos
4162,beleza_saude,sao paulo,SP,189361.87,1
6559,cama_mesa_banho,sao paulo,SP,170705.41,2
22694,relogios_presentes,sao paulo,SP,165897.46,3
15805,informatica_acessorios,sao paulo,SP,144635.68,4
12371,esporte_lazer,sao paulo,SP,144533.26,5
25251,utilidades_domesticas,sao paulo,SP,104820.45,6
18472,moveis_decoracao,sao paulo,SP,100033.05,7
22566,relogios_presentes,rio de janeiro,RJ,93028.74,8
3993,beleza_saude,rio de janeiro,RJ,86558.34,9
6402,cama_mesa_banho,rio de janeiro,RJ,77871.29,10


## Top Categorias Populares

In [7]:
# Remove registros duplicados
pedidos_unicos_df = df.drop_duplicates(['product_category_name','order_purchase_timestamp', 'customer_id'])

In [8]:
# Encontra o número de pedidos e o número de pedidos únicos de cada categoria
itens_mais_populares_df = pedidos_unicos_df.groupby(['product_category_name', 'customer_city', 'customer_state']).agg({'order_purchase_timestamp':'count', 'customer_id':'nunique'})
itens_mais_populares_df.columns=['no_pedidos', 'no_clientes']

In [9]:
# Reset o index convertendo o produto em coluna
itens_mais_populares_df.reset_index(inplace=True)

In [10]:
O = itens_mais_populares_df['no_pedidos']
C = itens_mais_populares_df['no_clientes']
M = itens_mais_populares_df['no_clientes'].max()

### Categorias de produtos com um número alto de pedidos pode ser considerado os mais frequentes comprados

Para encontrar os itens mais populares, inclua o número de clientes comprados e forneça mais peso às categorias de produtos adquiridas por mais clientes

Ponderação no_pedido (P) = O * (C / M)

O = no_pedidos

C = no_clientes que comprou algo da categoria de produto

M = Nº máximo de clientes que realizaram transações em todo o período

In [11]:
itens_mais_populares_df['no_pedidos_ponderado'] = O * (C / M)

In [12]:
# Rank de produtos de acordo com o número de pedidos ponderado
itens_mais_populares_df['rank_popularidade'] = itens_mais_populares_df['no_pedidos_ponderado'].rank(method='min',ascending=False).astype(int)

In [13]:
itens_mais_populares_df.sort_values('rank_popularidade',ascending=True).head(20)

,product_category_name,customer_city,customer_state,no_pedidos,no_clientes,no_pedidos_ponderado,rank_popularidade
6559,cama_mesa_banho,sao paulo,SP,1653,1653,1653.000000,1
4162,beleza_saude,sao paulo,SP,1533,1533,1421.711434,2
12371,esporte_lazer,sao paulo,SP,1266,1266,969.604356,3
25251,utilidades_domesticas,sao paulo,SP,1107,1107,741.348457,4
15805,informatica_acessorios,sao paulo,SP,1004,1004,609.810042,5
18472,moveis_decoracao,sao paulo,SP,998,998,602.543255,6
22694,relogios_presentes,sao paulo,SP,778,778,366.173019,7
6402,cama_mesa_banho,rio de janeiro,RJ,726,726,318.860254,8
5138,brinquedos,sao paulo,SP,610,610,225.105868,9
24030,telefonia,sao paulo,SP,570,570,196.551724,10


In [14]:
# Merge dos Ranks de Categoria de produtos mais Vendidos e dos mais Populares
categorias_rankings_df = pd.merge(top_itens_df, itens_mais_populares_df,how='inner',on='product_category_name')

In [15]:
categorias_rankings_df.columns

Index(['product_category_name', 'customer_city_x', 'customer_state_x', 'price',
       'rank_top_vendidos', 'customer_city_y', 'customer_state_y',
       'no_pedidos', 'no_clientes', 'no_pedidos_ponderado',
       'rank_popularidade'],
      dtype='object')

In [16]:
categorias_rankings_df = categorias_rankings_df.drop(["customer_city_x", "customer_state_x"], axis=1)

In [17]:
categorias_rankings_df.columns = [col.replace("_y", "") for col in categorias_rankings_df.columns]

In [18]:
# Filtra apenas as colunas 'product_category_name','rank_top_vendidos','rank_popularidade', 'customer_city', 'customer_state'
categorias_rankings_df = categorias_rankings_df[['product_category_name','rank_top_vendidos','rank_popularidade', 'customer_city', 'customer_state']]

In [19]:
categorias_rankings_df.head()

,product_category_name,rank_top_vendidos,rank_popularidade,customer_city,customer_state
0,agro_industria_e_comercio,5503,9567,alto piquiri,PR
1,agro_industria_e_comercio,5503,9567,americana,SP
2,agro_industria_e_comercio,5503,9567,amparo,SP
3,agro_industria_e_comercio,5503,9567,andradas,MG
4,agro_industria_e_comercio,5503,9567,andradina,SP


In [20]:
# Salva o dataset
categorias_rankings_df.to_csv('rankings_categoria.csv',index=False)

In [21]:
#Create a Pickle (.pkl) file with the Ranking dataframe
pickle.dump(categorias_rankings_df, open('prod_ranking_modelo.pkl','wb'))